In [11]:
from pathlib import Path

import pandas as pd
import pygwalker as pyg
from pandas import DataFrame

csv_files = list(Path("../.tmp/btransactions").glob("*.csv"))
df_list = [pd.read_csv(file, skiprows=9, delimiter=";") for file in csv_files]
transactions = pd.concat(df_list, ignore_index=True)

In [12]:
def clean_data(transactions: DataFrame) -> DataFrame:
    # Remove columns with no values
    transactions = transactions.dropna(axis=1, how="all")
    # Remove column Abschlusszeit
    transactions = transactions.drop(columns=["Abschlusszeit"])
    # Create Buchungsmonat column from Valutadatum month
    transactions["Buchungsmonat"] = pd.to_datetime(transactions["Buchungsdatum"]).dt.month
    # Create Buchungsjahr column from Buchungsdatum year
    transactions["Buchungsjahr"] = pd.to_datetime(transactions["Buchungsdatum"]).dt.year
    # Create Week column from Buchungsdatum week
    transactions["Buchungswoche"] = pd.to_datetime(transactions["Buchungsdatum"]).dt.isocalendar().week
    # Move columns
    cols = transactions.columns.tolist()
    valutadatum_index = cols.index("Valutadatum")
    cols.insert(valutadatum_index + 1, cols.pop(cols.index("Buchungsjahr")))
    cols.insert(valutadatum_index + 2, cols.pop(cols.index("Buchungsmonat")))
    cols.insert(valutadatum_index + 3, cols.pop(cols.index("Buchungswoche")))
    # Sort by column: "Buchungsdatum" (ascending)
    transactions = transactions.sort_values(["Buchungsdatum"])
    return transactions[cols]

transactions_clean = clean_data(transactions.copy())

transactions_clean.to_excel("../.tmp/transactions_clean.xlsx", index=False)



In [8]:
walker = pyg.walk(transactions_clean)

Box(children=(HTML(value='\n<div id="ifr-pyg-000630d6a85ff679aNIJjxkwFXGBVAiR" style="height: auto">\n    <hea…